**12 Days of Demos**
# 🎅 North Pole Vector Search System 🎄

Every year, Santa receives **millions of letters** from children around the world. Each letter is unique - some children are polite and formal, others are casual and excited. Some ask for creative gifts like art supplies, while others want the latest tech gadgets. Some letters overflow with holiday spirit, while others are short and to the point. How can Santa's elves quickly find similar letters, identify trends, and match children with the perfect gifts? This notebook creates a **semantic search system** using AI-powered vector embeddings. Instead of just matching keywords, our system understands the **meaning** behind each letter:

* **Personality & Tone**: Is the child grateful? Excited? Polite? Casual?
* **Gift Preferences**: Creative items? Tech gadgets? Toys? Unusual requests?
* **Christmas Spirit**: How much holiday enthusiasm does the letter show?
* **Writing Style**: Formal or informal? Detailed or brief?


---

### 🦌 Step 1: Configuration

The configuration settings below are where the demo will load and create data. You may choose to optionally change the settings if you prefer the demo use a different catalog or schema.

👇 **Optionally update the cell below, then run it!**

In [0]:
# TODO: Optionally update these values for your environment
TARGET_CATALOG = "12daysofdemos"
TARGET_SCHEMA = "raw_data"
TARGET_VOLUME = "raw_data_volume"
TARGET_BRONZE_SCHEMA = "bronze_data"  # Schema for the cloned table
DEFAULT_TABLE_NAME_BASE = "holiday_letters_canada"
CLONED_TABLE_NAME_BASE = "bronze_holiday_letters_canada"

print(f"✅ Configuration loaded")

In [0]:
%run "../00-init/load-data"

**Why the timestamp?** 

Each time you run this notebook, it creates a fresh endpoint and index with a unique name. This prevents conflicts if you're experimenting or running multiple versions! Tip: In production, you'd use fixed names and update existing endpoints rather than creating new ones each time.

In [0]:
import time
import datetime
from databricks.vector_search.client import VectorSearchClient

# Derived configuration - built from the values above
TARGET_TABLE_NAME = f"{TARGET_CATALOG}.{TARGET_SCHEMA}.{DEFAULT_TABLE_NAME_BASE}"
CLONED_TABLE_NAME = f"{TARGET_CATALOG}.{TARGET_BRONZE_SCHEMA}.{CLONED_TABLE_NAME_BASE}"

# Create unique names with timestamp for this magical vector search system
timestamp = str(int(time.time()))
endpoint_name = f'santa_letters_endpoint_{timestamp}'
index_name = f'{TARGET_CATALOG}.{TARGET_BRONZE_SCHEMA}.santa_letters_index_{timestamp}'

In [0]:
# Create the bronze schema if it doesn't exist
spark.sql(f"""
CREATE SCHEMA IF NOT EXISTS {TARGET_CATALOG}.{TARGET_BRONZE_SCHEMA}
""")

print(f"✅ Schema ready: {TARGET_CATALOG}.{TARGET_BRONZE_SCHEMA}")

### 🖌️ Step 2: Create Vector Search Endpoint

⚠️ **IMPORTANT**: Spinning up a Vector Search endpoint can take 20-30 minutes to provision!

This endpoint will power our semantic search system, allowing Santa's elves to find letters based on:
* **Personality traits**: grateful, excited, polite, enthusiastic
* **Gift categories**: creative gifts, tech gadgets, toys, outdoor items
* **Christmas spirit**: holiday enthusiasm, festive language, kindness
* **Special requests**: unusual items, specific brands, multiple wishes

In [0]:
# Initialize Vector Search Client
vsc = VectorSearchClient(disable_notice=True)

print(f"🎄 Creating new endpoint: {endpoint_name}")
print(f"🎄 Creating new index: {index_name}")
print("\n⏳ This may take 20-30 minutes... Time for some hot cocoa! ☕\n")

# Create the vector search endpoint and wait for it to be online
try:
    endpoint = vsc.create_endpoint_and_wait(
        name=endpoint_name, 
        endpoint_type='STANDARD',
        verbose=True,
        timeout=datetime.timedelta(seconds=3600)
    )
    print(f"\n✅ Successfully created endpoint: {endpoint_name}")
except Exception as e:
    print(f"\n❌ Error creating endpoint: {e}")
    raise

### 📰 Step 2: Creating the Vector Search Index

Now that our endpoint is ready, we need to create an **index** - think of it as Santa's magical filing system that organizes all the letters by their meaning!

**1. Clone the Source Table**
* We create a copy of the original letters table
* This protects the original data from any changes
* Enables Change Data Feed (CDF) for real-time updates

In [0]:
# Clone the source table to avoid affecting the original
spark.sql(f"""
CREATE OR REPLACE TABLE {CLONED_TABLE_NAME} CLONE {TARGET_TABLE_NAME}
""")

**2. Enable Change Data Feed**
* Tracks any changes to the table (new letters, updates)
* Allows the index to stay synchronized automatically
* Essential for production systems that receive new letters daily

In [0]:
# Enable Change Data Feed (CDF) on the cloned table
spark.sql(f"""
ALTER TABLE {CLONED_TABLE_NAME}
SET TBLPROPERTIES (delta.enableChangeDataFeed = true)
""")

**3. Create the Index**
* **Primary Key**: `name` (each child's name identifies their letter)
* **Embedding Source**: `letter` column (the full text we want to search)
* **Embedding Model**: `databricks-bge-large-en` (the AI that understands the text)
* **Pipeline Type**: `TRIGGERED` (we control when to update the index)

*Why the letter column?*

The `letter` column contains the complete text of each child's letter, including:
* Personality and tone
* Gift requests and preferences  
* Christmas spirit and enthusiasm
* Writing style (formal vs casual)

By embedding this full text, we capture all the nuances that make each letter unique!

⚠️ **Note**: The index creation starts immediately, but building embeddings for all letters takes a few minutes.

In [0]:
# Create the vector search index using BGE embeddings on the 'letter' column
try:
    index = vsc.create_delta_sync_index(
        endpoint_name=endpoint_name,
        index_name=index_name,
        source_table_name=CLONED_TABLE_NAME,
        pipeline_type="TRIGGERED",
        primary_key="name",
        embedding_source_column='letter',
        embedding_model_endpoint_name='databricks-bge-large-en'
    )
    print(f"\n✅ Successfully created index: {index_name}")
    print("\n🎉 Vector search setup complete!")
    print(f"\n🔌 Endpoint: {endpoint_name}")
    print(f"📇 Index: {index_name}")
    print("\n🎅 The elves can now search for letters semantically!")
except Exception as e:
    print(f"\n❌ Error creating index: {e}")
    raise

### 🧪 Step 3: Test Semantic Search

Time to put our North Pole Vector Search to the test! We'll search for letters based on:

* **Personality & tone**: grateful, excited, polite, casual
* **Gift preferences**: creative gifts, tech gadgets, unusual requests
* **Christmas spirit**: holiday enthusiasm, festive language

Describe what you're looking for in natural language, and the AI finds matching letters - even if the exact words aren't in the text!

In [0]:
# Wait for the index to finish building embeddings
print(f"🎄 Testing vector search with index: {index_name}")
print("⏳ Adding a 100-second buffer to make sure the index is up and running...")
print("🎅 Perfect time to check your nice list twice!\n")
time.sleep(100)
print("✅ Ready to search!")

**Test 1**: Find letters from grateful children asking for creative gifts

In [0]:
print("🔍 Searching for: Grateful children asking for creative gifts...\n")

try:
    results1 = spark.sql(f"""
    SELECT 
        search_score,
        name,
        city,
        province,
        letter,
        gifts
    FROM VECTOR_SEARCH(
        index => '{index_name}',
        query_text => 'grateful thankful children asking for creative artistic unique gifts',
        num_results => 5
    )
    ORDER BY search_score DESC
    """)
    
    print("✅ Vector search is working!")
    print("🎄 Top 5 letters from grateful children with creative requests:\n")
    display(results1)
    
except Exception as e:
    print(f"")
    print(f"⚠️"*20)
    print(f"⚠️ Vector search not ready yet  ⚠️")
    print(f"⚠️"*20)
    print(f"")
    print("🕒 The index is still building embeddings. Wait a few more minutes and then run this cell again!")


**What the AI Found**:
* Letters expressing gratitude (even without the exact word "grateful")
* Requests for creative items like art supplies, LEGO sets, craft materials
* Children who took time to be polite and appreciative


Notice how the search found letters that express gratitude in different ways - "thank you", "appreciate", "grateful", or just a polite tone. That's semantic search in action!

**Test 2**: Find letters with high Christmas spirit and unusual requests

In [0]:
# Test 2: Find letters with high Christmas spirit and unusual requests
print("🔍 Searching for: Letters with high Christmas spirit and unusual requests...\n")

try:
    results2 = spark.sql(f"""
    SELECT 
        search_score,
        name,
        city,
        province,
        letter,
        gifts
    FROM VECTOR_SEARCH(
        index => '{index_name}',
        query_text => 'excited enthusiastic festive Christmas spirit unusual unique interesting gift requests',
        num_results => 5
    )
    ORDER BY search_score DESC
    """)
    
    print("🎄 Top 5 letters with high Christmas spirit and unusual requests:\n")
    display(results2)
    
except Exception as e:
    print(f"⚠️ Vector search not ready yet: {e}")


**What the AI Found**:
* Letters with high energy and excitement ("hyped", "can't wait", exclamation marks!!!)
* Unusual gift requests like magnetic levitating globes, solar-powered chargers
* Strong Christmas spirit and festive language

The AI understands enthusiasm through writing style, not just words. It picks up on tone, punctuation, and energy level!

---


🚨 Important: To systematically turn off **all** of your Vector Search endpoints, run the cell below:

In [0]:
from databricks.sdk import WorkspaceClient

# Initialize the Workspace Client
w = WorkspaceClient()

# List all vector search endpoints
print("Fetching all vector search endpoints...")
endpoints = list(w.vector_search_endpoints.list_endpoints())
print(f"Found {len(endpoints)} endpoint(s)\n")

all_indexes = []

if not endpoints:
    print("No vector search endpoints found.")
else:
    # For each endpoint, list all indexes
    for endpoint in endpoints:
        endpoint_name = endpoint.name
        print(f"Checking endpoint: {endpoint_name}")
        
        try:
            # List all indexes for this endpoint
            indexes = list(w.vector_search_indexes.list_indexes(endpoint_name=endpoint_name))
            print(f"  Found {len(indexes)} index(es)")
            
            for index in indexes:
                index_name = index.name
                print(f"    - {index_name}")
                all_indexes.append(index_name)
        except Exception as e:
            print(f"  Error listing indexes for endpoint {endpoint_name}: {str(e)}")
        
        print()
    
    print(f"\n{'='*60}")
    print(f"Total indexes found: {len(all_indexes)}")
    print(f"{'='*60}")
    
    if all_indexes:
        print("\nTo delete these indexes, run the next cell.")

# Get all indexes to delete
print("Fetching all vector search indexes...")
endpoints = list(w.vector_search_endpoints.list_endpoints())

total_deleted = 0
for endpoint in endpoints:
    endpoint_name = endpoint.name
    
    try:
        indexes = list(w.vector_search_indexes.list_indexes(endpoint_name=endpoint_name))
        
        for index in indexes:
            index_name = index.name
            print(f"Deleting index: {index_name}")
            try:
                w.vector_search_indexes.delete_index(index_name=index_name)
                print(f"  ✓ Successfully deleted: {index_name}")
                total_deleted += 1
            except Exception as e:
                print(f"  ✗ Error deleting {index_name}: {str(e)}")
    except Exception as e:
        print(f"Error processing endpoint {endpoint_name}: {str(e)}")

print(f"\n{'='*60}")
print(f"Total indexes deleted: {total_deleted}")
print(f"{'='*60}")